In [2]:
import pandas as pd
import requests
from sqlalchemy import create_engine, text
import json

# Base URL and JSON files
base_url = 'https://raw.githubusercontent.com/Shyamalasridar/exa-data-eng-assessment/main/data/'
json_files = ['Chase54_Crooks415_e5b7d947-da2b-9cf4-12a0-9eb2cf735330.json']

# Lists to hold patient and resource data
patient_data_list = []
other_resources_list = []

# Process each JSON file
for json_file in json_files:
    response = requests.get(base_url + json_file)
    data = response.json()
    
    for entry in data['entry']:
        resource = entry['resource']
        if resource['resourceType'] == 'Patient':
            name = ''
            if 'name' in resource and resource['name']:
                name_parts = [part for part in resource['name'][0]['given']]
                name_parts.append(resource['name'][0]['family'])
                name = ' '.join(name_parts)
                print(f"patient id in patientinfo table '{resource['id']}'.")
            patient_data = {
                'fullUrl': entry['fullUrl'],
                'resourceType': resource['resourceType'],
                'id': resource['id'],
                'name': name,
                'gender': resource.get('gender', ''),
                'birthDate': resource.get('birthDate', ''),
                'deceasedDateTime': resource.get('deceasedDateTime', ''),
                'maritalStatus': resource.get('maritalStatus', {}).get('text', ''),
                'city': resource['address'][0].get('city', '') if 'address' in resource and resource['address'] else '',
                'state': resource['address'][0].get('state', '') if 'address' in resource and resource['address'] else '',
                'country': resource['address'][0].get('country', '') if 'address' in resource and resource['address'] else ''
            }
            patient_data_list.append(patient_data)
        else:
            patient_id = resource.get('subject', {}).get('reference', '').split('/')[-1]
            if patient_id:  # Only add if patient_id is not empty
                resource_data = {
                    'fullUrl': entry['fullUrl'],
                    'resourceType': resource['resourceType'],
                    'id': resource['id'],
                    'patientId': patient_id,
                    'details': json.dumps(resource)
                }
                print(f"patient id in resourceinfo table '{resource_data['patientId']}'.")
                print(f"resourceType in resourceinfo table '{resource_data['resourceType']}'.")
                other_resources_list.append(resource_data)

# Create DataFrames
patient_df = pd.DataFrame(patient_data_list)
resources_df = pd.DataFrame(other_resources_list)

# SQL Server connection
server = 'AMSAS'
database = 'EMIS'
connection_string = f'mssql+pyodbc://{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

# Function to delete existing records
def delete_existing_records(engine, patient_id):
    with engine.connect() as connection:
        delete_query = text("DELETE FROM PatientInformation WHERE id = :id")
        connection.execute(delete_query, {'id': patient_id})

try:
    patient_table_name = 'PatientInformation'
    resources_table_name = 'ResourcesInformation'

    # Insert patient data
    for index, row in patient_df.iterrows():
        delete_existing_records(engine, row['id'])
        row.to_frame().T.to_sql(patient_table_name, engine, if_exists='append', index=False)

    # Insert resource data
    for index, row in resources_df.iterrows():
        if row['patientId'] in patient_df['id'].values:  # Ensure patientId exists in PatientInformation
            row.to_frame().T.to_sql(resources_table_name, engine, if_exists='append', index=False)

    print(f"Data has been successfully saved to the SQL Server database '{database}', tables '{patient_table_name}' and '{resources_table_name}'.")
except Exception as e:
    print(f"Error saving data to SQL Server: {e}")
finally:
    engine.dispose()


patient id in patientinfo table 'e5b7d947-da2b-9cf4-12a0-9eb2cf735330'.
patient id in resourceinfo table 'urn:uuid:e5b7d947-da2b-9cf4-12a0-9eb2cf735330'.
resourceType in resourceinfo table 'Encounter'.
patient id in resourceinfo table 'urn:uuid:e5b7d947-da2b-9cf4-12a0-9eb2cf735330'.
resourceType in resourceinfo table 'DiagnosticReport'.
patient id in resourceinfo table 'urn:uuid:e5b7d947-da2b-9cf4-12a0-9eb2cf735330'.
resourceType in resourceinfo table 'DocumentReference'.
patient id in resourceinfo table 'urn:uuid:e5b7d947-da2b-9cf4-12a0-9eb2cf735330'.
resourceType in resourceinfo table 'Claim'.
patient id in resourceinfo table 'urn:uuid:e5b7d947-da2b-9cf4-12a0-9eb2cf735330'.
resourceType in resourceinfo table 'ExplanationOfBenefit'.
patient id in resourceinfo table 'urn:uuid:e5b7d947-da2b-9cf4-12a0-9eb2cf735330'.
resourceType in resourceinfo table 'Encounter'.
patient id in resourceinfo table 'urn:uuid:e5b7d947-da2b-9cf4-12a0-9eb2cf735330'.
resourceType in resourceinfo table 'Diagnost